In [1]:
import numpy as np 
import scipy.io as io 


import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

from IPython.display import clear_output
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
data = io.loadmat('Data/Train_Test.mat')

u_train = data['u_train']
y_train = data['y_train']
s_train =  data['s_train']

u_test = data['u_test']
y_test = data['y_test']
s_test = data['s_test']

In [4]:
class CustomDataset(Dataset):
    def __init__(self,dataset_branch, dataset_trunk, dataset_sol):
        self.dataset1 = dataset_branch
        self.dataset2 = dataset_trunk
        self.dataset3 = dataset_sol

    def __len__(self):
        return len(self.dataset1)

    def __getitem__(self, index):
        data1 = self.dataset1[index]
        data2 = self.dataset2[index]
        data3 = self.dataset3[index]
        return data1, data2, data3

In [5]:
### builinding the model 
class DeepOnet(nn.Module):
    def __init__(self, depth =2 , width = 40, branch_in=100, trunk_in=20, out_put_dim = 20):
        super(DeepOnet, self).__init__()
        self.depth = depth

        branch_neurons = [branch_in] + [width]*depth + [out_put_dim]
        trunk_neurons = [trunk_in] + [width]*depth + [out_put_dim]


        self.branch_net = self.net_creation(depth, branch_neurons)
        self.trunk_net = self.net_creation(depth, trunk_neurons)

        self.b = torch.nn.parameter.Parameter(torch.tensor(0.0))

    def net_creation(self, depth, neurons):
        layers = []
        for i in range(depth):
            if i < depth -1: 
                layer = nn.Sequential(
                    nn.Linear(neurons[i], neurons[i+1]), 
                    nn.Tanh()

                )
            else:
                layer = nn.Sequential(
                    nn.Linear(neurons[i+1], neurons[i+2])
                )

            layers.append(layer)
        return nn.Sequential(*layers)
    
    def forward(self, u, y):
        branch_out = self.branch_net(u)
        trunk_out = self.trunk_net(y)
        out = torch.einsum("bi, bi->bi", branch_out, trunk_out)
        out += self.b
        return out


In [6]:
## Train Function

def train(model, data_loader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    for batch in data_loader:
        batch = [tensor.to(device) for tensor in batch]
        u, y, s = batch 

        s_pred = model(u, y)
        loss = criterion(s_pred, s)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss/len(data_loader)

    return avg_loss

## Test Function 

def test(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    for batch in data_loader:
        batch = [tensor.to(device) for tensor in batch]
        u, y, s = batch
        with torch.no_grad():
            s_pred = model(u, y)
            loss = criterion(s_pred, s)

        total_loss += loss.item()

    avg_loss = total_loss/len(data_loader)

    return avg_loss    


### Combine functions 
def training(EPOCHS, train_dataloader, test_dataloader, d, w):

    np.random.seed(0)
    model = DeepOnet(branch_in=100, trunk_in=20, depth=d, width=w, out_put_dim=20).to(device=device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

    train_losses = []
    test_losses = []
    for epoch in range(EPOCHS):
        train_loss = train(model=model, data_loader=train_dataloader, criterion=criterion, optimizer=optimizer)
        train_losses.append(train_loss)


        test_loss = test(model=model, data_loader=test_dataloader, criterion=criterion)
        test_losses.append(test_loss)

        if epoch %20 == 0:
            print(f'D{d}W{w} || Epoch: {epoch} || Train Loss: {train_loss: .12f} || Test Loss: {test_loss: .12f}')

        if epoch %200 == 0:
            clear_output(wait=True)

    return model, train_losses, test_losses

In [7]:
BATCH_SIZE = 500 
NUM_WORKRS = 6

train_dataset = CustomDataset(u_train, y_train, s_train)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKRS)

test_dataset = CustomDataset(u_test, y_test, s_test)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE*10, num_workers=NUM_WORKRS)

depths = [2, 3, 4]
widths = [10, 160, 2560]
EPOCHS = 10_000


for depth, width in zip(depths, widths):
    model, train_losses, test_losses = training(EPOCHS=EPOCHS,
                                                train_dataloader=train_dataloader,
                                                test_dataloader=test_dataloader, 
                                                d = depth, 
                                                w = width)
    saved_folder = f'Results/D{depth}W{width}'

    if not os.path.exists(saved_folder):
        os.makedirs(saved_folder)

    train_losses = np.array(train_losses)
    test_losses = np.array(test_losses)

    np.save(f'{saved_folder}/train_losses.npy', train_losses)
    np.save(f'{saved_folder}/test_losses.npy', test_losses)
    torch.save(model, f'{saved_folder}/model.pth')


D2W10 || Epoch: 1220 || Train Loss:  0.000367829644 || Test Loss:  0.000365100276
D2W10 || Epoch: 1240 || Train Loss:  0.000886657975 || Test Loss:  0.000831594761
D2W10 || Epoch: 1260 || Train Loss:  0.000175025571 || Test Loss:  0.000141442732
D2W10 || Epoch: 1280 || Train Loss:  0.000245675033 || Test Loss:  0.000274789934
D2W10 || Epoch: 1300 || Train Loss:  0.000267813917 || Test Loss:  0.000366630893
D2W10 || Epoch: 1320 || Train Loss:  0.000241190952 || Test Loss:  0.000337971766
